# SPML Project

##### Using graph attention to learn semantic in point cloud 3D manmade structure and keep only useful/significant points

In [12]:
import numpy as np
from scipy.spatial import KDTree
import networkx as nx
import open3d as o3d
import os

## 0. Load datasets

## 1. Convert points cloud to graph

Load semantic 3D file

In [13]:
def load_semantic3d_txt(path):
    data = np.loadtxt(path)
    points = data[:, 0:3]      # x, y, z
    features = data[:, 3:7]    # intensity, r, g, b
    return points, features

Radius estimation

In [14]:
def estimate_radius(points, k=1, scale=2.0):
    tree = KDTree(points)
    dists, _ = tree.query(points, k=k+1)
    return np.mean(dists[:, k]) * scale

Graph construction

In [15]:
def prune_to_k_neighbors(graph, k):
    for node in graph.nodes():
        edges = [(node, n, graph[node][n]['weight']) for n in graph[node]]
        if len(edges) > k:
            edges.sort(key=lambda x: x[2])
            graph.remove_edges_from([(e[0], e[1]) for e in edges[k:]])


In [16]:
def build_radius_graph(points, radius, max_neighbors=None):
    kdtree = KDTree(points)
    graph = nx.Graph()

    for i in range(len(points)):
        graph.add_node(
            i,
            pos=points[i].tolist()
        )

    for i, j in kdtree.query_pairs(radius):
        dist = np.linalg.norm(points[i] - points[j])
        graph.add_edge(i, j, weight=dist)

    if max_neighbors is not None:
        prune_to_k_neighbors(graph, max_neighbors)

    return graph


Save graph in local

In [17]:
def save_graph(graph, path):
    nx.write_gpickle(graph, path)


Load it

In [18]:
def load_graph(path):
    return nx.read_gpickle(path)


Load files and convert it into graph

In [19]:
def process_files(
    file_list,
    input_dir,
    output_dir,
    max_neighbors=10,
    radius_scale=2.0
):
    os.makedirs(output_dir, exist_ok=True)

    for filename in file_list:
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(
            output_dir,
            filename.replace(".txt", ".graph")
        )

        if os.path.exists(output_path):
            print(f"[SKIP] {filename} (graph already exists)")
            continue

        print(f"[PROCESS] {filename}")

        # 1. Load data
        points, features = load_semantic3d_txt(input_path)

        # 2. Estimate radius
        radius = estimate_radius(points, k=1, scale=radius_scale)

        # 3. Build graph
        graph = build_radius_graph(
            points,
            radius=radius,
            max_neighbors=max_neighbors
        )

        # 4. Attach node features
        for i in range(len(points)):
            graph.nodes[i]["feat"] = np.concatenate(
                [points[i], features[i]]
            ).tolist()

        # 5. Save graph
        save_graph(graph, output_path)


In [21]:
file_list = [
    "MarketplaceFeldkirch_Station4_rgb_intensity-reduced.txt",
    "StGallenCathedral_station6_rgb_intensity-reduced.txt",
]

process_files(
    file_list=file_list,
    input_dir="dataset/raw_txt",
    output_dir="dataset/graphs",
    max_neighbors=10,
    radius_scale=2.0
)


[PROCESS] MarketplaceFeldkirch_Station4_rgb_intensity-reduced.txt


AttributeError: module 'networkx' has no attribute 'write_gpickle'